# ICOER Text Analysis Notebook
Análise de frases com vetorização POS, cálculo da métrica ICOER (normalizada) e exportação CSV.

In [ ]:
!pip install -q spacy pandas matplotlib ipywidgets
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Carregar modelo SpaCy
nlp = spacy.load("pt_core_news_sm")

In [ ]:
# Caixa de entrada de texto interativa
text_input = widgets.Textarea(
    value='A IA deve sempre agir com coerência e verdade. O índice de coerência informacional guia decisões éticas.',
    placeholder='Digite suas frases separadas por ponto final...',
    description='Frases:',
    layout=widgets.Layout(width='100%', height='100px')
)
display(text_input)

In [ ]:
# Separar frases com base em ponto final
frases = [f.strip() for f in text_input.value.split('.') if f.strip()]
frases

In [ ]:
# Análise de tags e vetores POS
data = []
for frase in frases:
    doc = nlp(frase)
    tags = [token.pos_ for token in doc]
    tag_counts = dict(Counter(tags))
    data.append({
        "frase": frase,
        "n_palavras": len(doc),
        "tags": tag_counts
    })
df = pd.DataFrame(data)

# Vetorização POS
def vetor_pos(tags_dict):
    base = ['NOUN', 'VERB', 'ADJ', 'ADV', 'ADP', 'PROPN']
    total = sum(tags_dict.values())
    return [tags_dict.get(pos, 0)/total if total > 0 else 0 for pos in base]

df['vetor'] = df['tags'].apply(vetor_pos)

In [ ]:
# Cálculo do ICOER com entropia normalizada
def entropia_normalizada(vetor):
    vetor = np.array(vetor)
    vetor = vetor[vetor > 0]
    ent = -np.sum(vetor * np.log2(vetor)) if len(vetor) > 0 else 0
    ent_max = np.log2(len(vetor)) if len(vetor) > 0 else 1
    return 1 - (ent / ent_max) if ent_max > 0 else 1

df['icoer'] = df['vetor'].apply(lambda v: round(entropia_normalizada(v), 4))
df[['frase', 'icoer']]

In [ ]:
# Exportação para CSV
df[['frase', 'n_palavras', 'tags', 'icoer']].to_csv("icoer_text_analysis.csv", index=False)
df